These analysis are based on https://cran.r-project.org/web/packages/VARsignR/vignettes/VARsignR-vignette.html#fnref2

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from statsmodels.tsa.vector_ar import var_model as sm
import numpy as np
from math import sqrt
from statsmodels.tsa.vector_ar import util

E:\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


first we need a pure cholesky decomposition version

In [2]:
data = pd.read_csv("test_data3.csv")
lags = 1

In [3]:
var = sm.VAR(endog=data.values)

In [4]:
result = var.fit(lags, trend="nc")

E:\Anaconda2\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]


first estimate unrestricted VAR, then distract A_td and sigma_td from the regression result, where  
$$\Sigma = E[uu^T]$$
$$u = Be$$
$$BB^T = \Sigma$$

for estimating VAR(p), we have to write the estimation ourselves

Then we use a cholesky decomposition to generate uncorrelated innovations, notice that we are not using the decomposition matrix as an ordering, but just to generate uncorrelated innovations, namely P

So the innovations according to the cholesky decomposition should be  
following Uhlig’s(2005) rejection method, we draw impulse vector a from unit sphere

Then we start the restricted impulse reponse analysis

here is a VAR(p) version

In [5]:
def irf(var_result, innovation_idx, larger_restrict_matrix, less_restrict_matrix, impulse_count, draw_count, restrict_period, verbose=False, plot=True):
    
    def get_sphere_rand():
        while True:
            trand = np.random.rand(len(sigma)) - 0.5
            return trand / sqrt(sum(trand**2))
    
    larger_restrict_array = larger_restrict_matrix[:, innovation_idx]
    less_restrict_array = less_restrict_matrix[:, innovation_idx]
    p, k = result.coefs.shape[0], result.coefs.shape[1]
    A_1 = result.coefs[0]
    sigma = result.sigma_u
    
    P = np.linalg.cholesky(sigma)
    B = np.linalg.inv(P)
    
    impulse_matrix = np.zeros((draw_count, A_1.shape[0], impulse_count))
    a_matrix = np.zeros((A_1.shape[0], draw_count))
    
    a = None
    for j in xrange(draw_count):
        if verbose:
            print j
        i = 0
        a = get_sphere_rand()
        while True:
            if i == impulse_count:
                break
            t_impulse = (A_1**(i + 1)).dot(B).dot(a)    
            if i < restrict_period and (t_impulse[larger_restrict_array] > 0.0).all() and (t_impulse[less_restrict_array] < 0.0).all():
                impulse_matrix[j, :, i] = t_impulse
                a_matrix[:, j] = a
                i += 1
            elif i >= restrict_period:
                impulse_matrix[j, :, i] = t_impulse
                a_matrix[:, j] = a
                i += 1
            else:
                a = -a # variance control method

                if i < restrict_period and (t_impulse[larger_restrict_array] > 0.0).all() and (t_impulse[less_restrict_array] < 0.0).all():
                    impulse_matrix[j, :, i] = t_impulse
                    a_matrix[:, j] = a
                    i += 1
                elif i >= restrict_period:
                    impulse_matrix[j, :, i] = t_impulse
                    a_matrix[:, j] = a
                    i += 1
                else:
                    i = 0
                    a = get_sphere_rand()
    
    if plot:
        print a_matrix.mean(0).shape
        fig = plt.figure(figsize=(20, 10))
        plt.plot(impulse_matrix.mean(0)[0, :], label="SRB")
        plt.plot(impulse_matrix.mean(0)[1, :], label="DXI")
        plt.plot(impulse_matrix.mean(0)[2, :], label="DXJ")
        plt.plot(impulse_matrix.mean(0)[3, :], label="DJM")
        plt.plot(impulse_matrix.mean(0)[4, :], label="ORC58")
        plt.plot(impulse_matrix.mean(0)[5, :], label="ORC62")
        plt.legend()
        plt.show()
    
    return a_matrix

In [6]:
larger_constraints = np.array(([True, True, True, False], [False, False, True, False], [False, True, True, False], [False, False, True, False], [False, False, False, True], [False, False, False, True]))
less_constraints = np.array(([False, False, False, False], [True, False, False, False], [True, False, False, False], [True, True, False, False], [True, False, False, False], [True, False, False, False]))

In [7]:
a = irf(result, 0, larger_constraints, less_constraints, 5, 200, 2, plot=False)
# 钢铁限产

In [8]:
a.mean(1)

array([ 0.53756044,  0.51072015,  0.22948202, -0.0474934 ,  0.1434727 ,
        0.03775386])

In [10]:
# 钢铁限产
fit_lag = 200
gt_result_lst = []

for i in xrange(len(data)):
    if i < fit_lag:
        continue
    print i
    this_data = this_data.iloc[i - fit_lag : i, :]
    var = sm.VAR(endog=data.values)
    result = var.fit(lags, trend="nc")
    
    a = irf(result, 0, larger_constraints, less_constraints, 5, 200, 2, plot=False)
    gt_result_lst.append(a.mean(1))

200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449


In [ ]:
gt_ar = np.array(gt_result_lst)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(gt_ar[0, :], label="0")
plt.plot(gt_ar[1, :], label="1")
plt.plot(gt_ar[2, :], label="2")
plt.plot(gt_ar[3, :], label="3")
plt.plot(gt_ar[4, :], label="4")
plt.plot(gt_ar[5, :], label="5")
plt.legend()
plt.show()

In [ ]:
# 焦化限产
fit_lag = 200
jh_result_lst = []

for i in xrange(len(data)):
    if i < fit_lag:
        continue
    this_data = data.iloc[i - fit_lag : i, :]
    var = sm.VAR(endog=data.values)
    result = var.fit(lags, trend="nc")
    
    a = irf(result, 1, larger_constraints, less_constraints, 5, 200, 2, plot=False)
    jh_result_lst.append(a.mean(1))

In [ ]:
jh_ar = np.array(jh_result_lst)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(jh_ar[0, :], label="0")
plt.plot(jh_ar[1, :], label="1")
plt.plot(jh_ar[2, :], label="2")
plt.plot(jh_ar[3, :], label="3")
plt.plot(jh_ar[4, :], label="4")
plt.plot(jh_ar[5, :], label="5")
plt.legend()
plt.show()

In [ ]:
# 钢铁需求
fit_lag = 200
gx_result_lst = []

for i in xrange(len(data)):
    if i < fit_lag:
        continue
    this_data = data.iloc[i - fit_lag : i, :]
    var = sm.VAR(endog=data.values)
    result = var.fit(lags, trend="nc")
    
    a = irf(result, 2, larger_constraints, less_constraints, 5, 200, 2, plot=False)
    gx_result_lst.append(a.mean(1))

In [ ]:
gx_ar = np.array(gx_result_lst)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(gx_ar[0, :], label="0")
plt.plot(gx_ar[1, :], label="1")
plt.plot(gx_ar[2, :], label="2")
plt.plot(gx_ar[3, :], label="3")
plt.plot(gx_ar[4, :], label="4")
plt.plot(gx_ar[5, :], label="5")
plt.legend()
plt.show()

In [ ]:
# 铁矿石涨
fit_lag = 200
tk_result_lst = []

for i in xrange(len(data)):
    if i < fit_lag:
        continue
    this_data = data.iloc[i - fit_lag : i, :]
    var = sm.VAR(endog=data.values)
    result = var.fit(lags, trend="nc")
    
    a = irf(result, 2, larger_constraints, less_constraints, 5, 200, 2, plot=False)
    tk_result_lst.append(a.mean(1))

In [ ]:
tk_ar = np.array(tk_result_lst)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(tk_ar[0, :], label="0")
plt.plot(tk_ar[1, :], label="1")
plt.plot(tk_ar[2, :], label="2")
plt.plot(tk_ar[3, :], label="3")
plt.plot(tk_ar[4, :], label="4")
plt.plot(tk_ar[5, :], label="5")
plt.legend()
plt.show()

In [ ]:
irf(result, 1, larger_constraints, less_constraints, 5, 200, 2)
# 焦化限产

In [ ]:
irf(result, 2, larger_constraints, less_constraints, 5, 200, 2)
# 钢铁需求

In [ ]:
irf(result, 3, larger_constraints, less_constraints, 5, 200, 2)
# 铁矿石涨